# Integrating an Action Tool in an Agent - SOLUTION

> **Learning Outcomes:**
> - Create an action tool with AutoGen.
> - Integrate the action tool with an agent.
> - Combine Observation and Action tools in an agent.
> - Understand the AutoGen Agent flow.

## Introduction

In this lab, we will implement an action tool that will allow the agent to take actions in the environment. This tool will allow the agent to interact with the world outside of the agent. This is a useful capacity for an agent to have, as it allows the agent to affect the world in a meaningful way.

### AutoGen Framework
[AutoGen](https://microsoft.github.io/autogen/stable/index.html) is "a framework for building AI agents and applications" that is developed by Microsoft. It is designed to be a flexible and extensible framework that can be used to build a wide range of AI applications.

We will be working with the `autogen-agentchat` component which is designed for building conversational single and multi-agent systems. This component is built on top of the `autogen` framework and provides a simple way to build agents that can communicate with each other.


### The Scenario
Our agent will be tasked with organizing a directory of files. The agent will be able to create directories and move files between directories. The agent will be able to take these actions based on the observations it makes of the current state of the directory.


## Getting Started
Let's start by installing the required libraries and setting the OpenAI API key. The OpenAI API key is required to access the OpenAI models.

Run the following cells to install the required libraries and set the OpenAI API key.


In [1]:
%pip install --upgrade pip setuptools wheel
%pip install tiktoken --only-binary=:all:

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install -qU \
    openai==1.95.* \
    autogen-agentchat==0.6.* \
    autogen-ext[openai]==0.6.*

import os
import getpass

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

Note: you may need to restart the kernel to use updated packages.


Enter your OpenAI API key:  ········


## Core

### Step 1 - Write Directory Observation Tool

In order to interact with its environment, its important for the agent to be able to observe the environment. In this case, the agent will be observing the directory structure. We need to write observation tools to do this.

Most Agentic libraries, including AutoGen, allow tools to be written as functions. These functions are then converted to tools by the library for easy integration with the agent. While the exact method of writing these functions may vary depending on the library, the general process is the same.

1. Write a function that takes the necessary inputs and returns the desired output.
2. Add type hints to the function to specify the types of the inputs and outputs.
3. Create a description of the function.


```python
def hello_world(name: str) -> str:
    return f"Hello, {name}!"
```

In the cell below, write a function that takes a path to a directory and returns a list of files and directories in that directory. Python's `pathlib` is the easiest way to work with paths in Python. You can use the `Path` class from the `pathlib` module to work with paths.

In [3]:
from pathlib import Path

# YOUR CODE HERE
def list_files(directory: str):
    return [str(f) for f in Path(directory).iterdir()]

A function on it's own is not a tool. It needs to be converted to a tool using the `autogen` library. This is done by using the `FunctionTool` class. This class takes the function and a description of the function as input and creates a tool that can be used by the agent.

```python
tool = FunctionTool(hello_world, description="Greets the person by name.")
await tool.run_json({"title": "John"})
```

In the cell below, create a `FunctionTool` for the your function and test it by running the tool directly on the "messy" directory. Save the tool to the variable `list_directory_tool`.

In [4]:
from autogen_core.tools import FunctionTool
from autogen_core import CancellationToken

# YOUR CODE HERE
#[SOLUTION]
list_directory_tool = FunctionTool(list_files, description="Content of a Wikipedia page given its title")

await list_directory_tool.run_json({"directory": "messy"}, CancellationToken())

['messy/Sales_Forecast_2024.xlsx',
 'messy/Regulatory_Update_Nov2023.docx',
 'messy/SocialMedia_Content_Plan.docx',
 'messy/2023_Annual_Report.pdf',
 'messy/Sales_Report_November.pdf',
 'messy/GDPR_Compliance_Checklist.xlsx',
 'messy/Tax_Return_2021_Final.pdf',
 'messy/Salary_Adjustment_Memo.docx',
 'messy/ProjectPlan_Alpha.pdf',
 'messy/VendorPayment_Ref12345.txt',
 'messy/Task_List_ProjectBeta.csv',
 'messy/Project_Budget_Tracking.xlsx',
 'messy/Patent_Application_2023.pdf',
 'messy/EmploymentContract_Doe.docx',
 'messy/Internal_Job_Posting_XYZ.pdf',
 'messy/CRM_Data_Export.csv',
 'messy/Q3_Budget_Draft.xlsx',
 'messy/Customer_Feedback_Survey.xlsx',
 'messy/Compliance_Report_Q3.xlsx',
 'messy/test.ipynb',
 'messy/Personnel_File_78901.zip',
 'messy/Marketing_Material_Brochure.pdf',
 'messy/Project_XYZ_CostAnalysis.csv',
 'messy/Compliance_Training_Slides.pptx',
 'messy/SalesProposal_ClientABC.pdf',
 'messy/Operational_Manual_2023.docx',
 'messy/Marketing_Campaign_Results.xlsx',
 'mess

### Step 2 - Write Action Tools

Now we need to write an action tool that will allow the agent to take actions in the environment. In this case, we will create three tools.

- `create_directory`
- `move_file`

It's important to limit the actions that the agent can take to prevent the agent from taking actions that could be harmful. In a real-world scenario, we would want to ensure that the agent can only access the directories that it is supposed to access. However, we will not be implementing this in this lab.

In the cell below, write two functions.

1. `create_directory`: This function should take a path for the new directory. The function should return the path to the new directory.
2. `move_file`: This function should take the path to a file and the path to a directory and move the file to the given directory. The function should return the path to the file after it has been moved.

Test the function as you did with the observation tool.

In [5]:
# YOUR CODE HERE
# [SOLUTION]

def create_directory(directory: str) -> Path:
    path = Path(directory)
    path.mkdir(exist_ok=True, parents=True)
    return path

def move_file(source: str, destination: str) -> Path:
    return Path(source).replace(destination)

create_directory("messy/test")
move_file("messy/2023_Annual_Report.pdf", "messy/test/2023_Annual_Report.pdf")

PosixPath('messy/test/2023_Annual_Report.pdf')

Now that the functions are defined, create `FunctionTool` objects for each of the functions. Save the tools to the variables `create_directory_tool` and `move_file_tool`.

In [6]:
# YOUR CODE HERE

# [SOLUTION]
create_directory_tool = FunctionTool(create_directory, description="Create a directory")
move_file_tool = FunctionTool(move_file, description="Move a file")

### Step 3 - Create an Agent

Now that we have all the tools we need, we need to integrate it with the agent. AutoGen's `AssistantAgent` class has the built-in capability to use a tool and respond based on it's output. However, it can only use one tool, once per task. To allow the the agent to use the tool multiple times, we can place it in a round-robin group chat by itself. This will allow the agent to continue working until it is complete.

Run the cell below to see how the agent uses the tool. This cell may take a while to run, as the agent will be working on the directory for a while.

<div class="alert alert-info">
    <h3>Note</h3>
    <p>The files in the `messy` directory are empty and don't contain any content. If you need to recreate the directory, an extra copy is stored inside the "solutions" directory. Simply delete the messy directory and <strong>copy</strong> the messy directory from solutions into its place.</p>
</div>

In [7]:
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.ui import Console
import textwrap

model_client = OpenAIChatCompletionClient(model="gpt-4.1-mini")
agent = AssistantAgent(
    name="directory_organizer",
    model_client=model_client,
    tools=[list_directory_tool, create_directory_tool, move_file_tool],
    system_message=textwrap.dedent("""
        You are an AI assistant that organizes files in a directory.

        When a user requests a directory to be organized, perform the following tasks:

        1. List the files in the directory.
        2. Create a new directory inside the given directory, named "Organized".
        3. If the user has not provided a list of categories, create a list of categories based on the file names.
        4. Create a subdirectory for each category.
        5. Move the files to their respective category subdirectories.

        Before completing the task, verify that all files have been organized by listing the contents of the original directory.
        Reply with TERMINATE when the task has been completed.
    """.strip()),
)

team = RoundRobinGroupChat([agent], termination_condition=TextMentionTermination("TERMINATE"))

await Console(team.run_stream(task="Organize `messy`"))

---------- TextMessage (user) ----------
Organize `messy`
---------- ToolCallRequestEvent (directory_organizer) ----------
[FunctionCall(id='call_FYgLlqD1pd8fH2XKyrZVFMBb', arguments='{"directory":"messy"}', name='list_files')]
---------- ToolCallExecutionEvent (directory_organizer) ----------
[FunctionExecutionResult(content="['messy/Sales_Forecast_2024.xlsx', 'messy/Regulatory_Update_Nov2023.docx', 'messy/SocialMedia_Content_Plan.docx', 'messy/Sales_Report_November.pdf', 'messy/GDPR_Compliance_Checklist.xlsx', 'messy/Tax_Return_2021_Final.pdf', 'messy/Salary_Adjustment_Memo.docx', 'messy/ProjectPlan_Alpha.pdf', 'messy/VendorPayment_Ref12345.txt', 'messy/Task_List_ProjectBeta.csv', 'messy/Project_Budget_Tracking.xlsx', 'messy/Patent_Application_2023.pdf', 'messy/EmploymentContract_Doe.docx', 'messy/test', 'messy/Internal_Job_Posting_XYZ.pdf', 'messy/CRM_Data_Export.csv', 'messy/Q3_Budget_Draft.xlsx', 'messy/Customer_Feedback_Survey.xlsx', 'messy/Compliance_Report_Q3.xlsx', 'messy/test

TaskResult(messages=[TextMessage(id='1a09593e-29b4-4d26-91cf-5dc7393fccdb', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 11, 23, 16, 9, 57, 346950, tzinfo=datetime.timezone.utc), content='Organize `messy`', type='TextMessage'), ToolCallRequestEvent(id='118e7ae2-82e1-4d19-8757-8b8da523e630', source='directory_organizer', models_usage=RequestUsage(prompt_tokens=259, completion_tokens=15), metadata={}, created_at=datetime.datetime(2025, 11, 23, 16, 9, 58, 99910, tzinfo=datetime.timezone.utc), content=[FunctionCall(id='call_FYgLlqD1pd8fH2XKyrZVFMBb', arguments='{"directory":"messy"}', name='list_files')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(id='16bed2e4-b22b-4710-a8b7-73db18b700ec', source='directory_organizer', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 11, 23, 16, 9, 58, 109781, tzinfo=datetime.timezone.utc), content=[FunctionExecutionResult(content="['messy/Sales_Forecast_2024.xlsx', 'messy/Regulatory_Update_N

Congratulations! You now have a working agent and an organized directory. You can now experiment with the agent by adding more files and directories to the `messy` directory and running the agent again.

## Bonus Challenge 1 - Split the Agent Tasks
It's often useful to split tasks across agents. Instead of having one agent that can do everything, we can have multiple agents that can do different things. This can make the agents more efficient and allow them to work together to solve complex problems.

For instance, we could have one agent that is responsible for deciding which category the file belongs in and another agent that is responsible for renaming the file to a standard format.

In the cell below, create a team of agents that can work together to organize the directory. The agents should be able to work together to organize the directory in the same way as the single agent.

In [8]:
# YOUR CODE HERE

## Bonus Challenge 2 - Add Security
In the current implementation, the agents can access any directory on the system. This is a security risk, as the agents could potentially access sensitive information. To prevent this, we need to add security to the agents.

The best way to do this is to restrict the tools themselves. We can do this by adding checks to the tools to ensure that the agent can only access the directories that it is supposed to access.

In the cell below, modify the tools to add the security checks. The tools should only allow the agent to access the `messy` directory and its subdirectories.

In [9]:
# YOUR CODE HERE